# Model: Desperado v0

**Model Parameters:**
* Ensemble model with simple averaging
* Including predictions from:
    * Foxhoud_v0
    * Deadcell_v0

**Note:**
* Updated: 2022-04-23

---
# Load data & libraries

In [1]:
# TOP VERSION
MODEL_NAME = "desperado"
VERSION = 0

In [2]:
# import dependencies
import pandas as pd
import gc
import json
from pathlib import Path
from numerapi import NumerAPI
from utils import (
    save_model,
    load_model,
    neutralize,
    get_biggest_change_features,
    ERA_COL,
    DATA_TYPE_COL,
    TARGET_COL,
)
from api_keys import (
    PUBLIC_ID,
    SECRET_KEY,
    DESPERADO_MODEL_ID
)

In [3]:
# instantiate numerai api
napi = NumerAPI(public_id=PUBLIC_ID, secret_key=SECRET_KEY)
current_round = napi.get_current_round()
print(f"Current round #: {current_round}")

Current round #: 313


In [4]:
# read predictions from models
foxhound_preds = pd.read_csv(f"predictions/dh_foxhound_v0_live_preds_{current_round}.csv")
deadcell_preds = pd.read_csv(f"predictions/dh_deadcell_v0_live_preds_{current_round}.csv")

print(f"""
    Foxhound predictions: {len(foxhound_preds)}
    Deadcell pedictions: {len(deadcell_preds)}
    """)


    Foxhound predictions: 5324
    Deadcell pedictions: 5324
    


---
# Create ensemble prediction

In [7]:
# model name
model_name = f"dh_{MODEL_NAME}_v{VERSION}"
model_name

'dh_desperado_v0'

In [15]:
# average predictions
desperado_df = foxhound_preds.merge(
    right=deadcell_preds, 
    how='inner', 
    on="id"
)

desperado_df.columns = ["id", "foxhound", "deadcell"]
desperado_df["prediction"] = (desperado_df["foxhound"] + desperado_df["deadcell"]) / 2
desperado_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5324 entries, 0 to 5323
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          5324 non-null   object 
 1   foxhound    5324 non-null   float64
 2   deadcell    5324 non-null   float64
 3   prediction  5324 non-null   float64
dtypes: float64(3), object(1)
memory usage: 208.0+ KB


In [19]:
# save ensemble prediction
desperado_preds = desperado_df[["id", "prediction"]].set_index("id")
desperado_preds.to_csv(f"predictions/{model_name}_live_preds_{current_round}.csv")

In [20]:
# submit live predictions
napi.upload_predictions(
    file_path=f"predictions/{model_name}_live_preds_{current_round}.csv", 
    model_id=DESPERADO_MODEL_ID
)

print(f"Submitted live prediction file '{model_name}_live_preds_{current_round}.csv'")

2022-04-23 20:06:44,040 INFO numerapi.base_api: uploading predictions...


Submitted live prediction file 'dh_desperado_v0_live_preds_313.csv'
